In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "../../../models/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_path).to("mps")
tokenizer = AutoTokenizer.from_pretrained(model_path)

/Users/bytedance/dev/data-juicer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


In [2]:
def inference(messages, model, tokenizer):
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

## Generate instructions

In [3]:
PROMPT_TEMPLATE = (
    """Generate a prompt the persona below might ask to an AI assistant:\n"""
)

system_prompt = "You are an AI assistant expert at simulating user interactions."
example_persona = "A behavioral economist or social psychologist interested in exploring strategies for influencing human decision-making and behavior change."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": PROMPT_TEMPLATE + example_persona},
]
print(inference(messages, model, tokenizer))

Sure, here's a prompt a behavioral economist or social psychologist might ask an AI assistant:

"How can we design interventions to effectively influence individual behaviors towards more sustainable practices, such as reducing plastic usage, while also considering factors like social norms and personal values?"


## Generate diverse text for pre-training and post-training

In [4]:
PROMPT_TEMPLATE = """Write a Quora post in the language, style, and personality of the following persona:\n"""

system_prompt = "You are an AI assistant specialized in writing posts for social media."
example_persona = "An economist specializing in education policy and research, likely with a background in quantitative analysis and public policy."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": PROMPT_TEMPLATE + example_persona},
]
print(inference(messages, model, tokenizer))

Absolutely! Let's dive into the fascinating world of how education policies impact our society. As an economist focusing on education, I often find myself analyzing data and figures to understand the ripple effects of educational reforms. Today, let’s explore one of the most crucial questions: How does early childhood education influence long-term economic outcomes?

Early childhood education programs have been shown to have a profound impact on children's development and future success. Research indicates that children who attend quality preschools tend to have better academic performance, higher graduation rates, and lower rates of criminal behavior as adults. This is not just about better test scores; it's about laying a solid foundation for lifelong learning and success.

But here’s where things get interesting – the return on investment in early childhood education is significant. Studies show that for every dollar spent on high-quality preschool programs, we can expect a return o

## Generate persona-specific problems

In [5]:
PROMPT_TEMPLATE = """Create a challenging math problem with the following persona:\n"""

system_prompt = "You are an AI assistant specialized in creating diverse but specific math problems. Just answer with your problem."
example_persona = "A behavioral economist or social psychologist interested in exploring strategies for influencing human decision-making and behavior change."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": PROMPT_TEMPLATE + example_persona},
]
print(inference(messages, model, tokenizer))

A behavioral economist is designing a study to explore how framing affects people's decisions regarding saving for retirement. Participants are given two options: 

Option A: "Save $500 per month for 30 years starting today."
Option B: "Do not save anything now, but if you wait 30 years, you will receive $1,240,000."

Participants are randomly assigned to one of these options. The economist then observes which option participants choose and their stated reasons for their choice.

The question is: If a participant chooses Option B, what strategy would be most effective in encouraging them to switch to Option A, considering psychological principles like loss aversion and mental accounting? Provide at least three strategies and explain why each might work.
